In [4]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [6]:
df = pd.read_csv("train_output.csv")

In [8]:
df.describe()

,Generation,Solution,Collected Targets,Fitness Score,Time
count,7919.000000,7919.000000,7919.000000,7919.000000,7919.000000
mean,20.845435,49.969188,0.661195,0.108597,47.431163
std,11.372912,28.576744,0.790508,0.020522,5.140654
min,1.000000,0.000000,0.000000,0.088442,38.250000
25%,11.000000,25.000000,0.000000,0.097825,44.216667
50%,21.000000,50.000000,1.000000,0.103101,46.383333
75%,30.500000,75.000000,1.000000,0.111561,49.183333
max,40.000000,99.000000,5.000000,0.336427,90.866667


In [26]:
max_value = df.describe().loc[['max']]['Fitness Score'].item()
df[df['Fitness Score'] == max_value]

,Generation,Solution,Collected Targets,Fitness Score,Time
4657,25,4,2,0.336427,90.866667
4756,25,4,2,0.336427,90.866667


In [12]:
def analyze_experiment(data_folder):
    """
    Analizuje wyniki eksperymentów w folderze i generuje wykresy.
    """
    # Przechodzi przez wszystkie pliki w folderze
    results = []
    for file in os.listdir(data_folder):
        if file.endswith(".csv"):
            df = pd.read_csv(os.path.join(data_folder, file))
            results.append(df)

    # Łączenie wyników w jedną tabelę
    full_data = pd.concat(results, axis=0, ignore_index=True)
    full_data['Generation'] = full_data['Generation'].astype(int)

    return full_data

In [14]:
def plot_fitness_evolution(data, output_path):
    """
    Generuje wykres ewolucji fitness.
    """
    plt.figure(figsize=(10, 6))
    generations = data['Generation'].unique()
    avg_fitness = data.groupby('Generation')['Fitness Score'].mean()
    max_fitness = data.groupby('Generation')['Fitness Score'].max()
    min_fitness = data.groupby('Generation')['Fitness Score'].min()
    std_fitness = data.groupby('Generation')['Fitness Score'].std()

    # Rysowanie średniego fitness z odchyleniem standardowym
    plt.plot(generations, avg_fitness, label="Średni fitness", color="blue")
    plt.fill_between(
        generations, 
        avg_fitness - std_fitness, 
        avg_fitness + std_fitness, 
        color="blue", alpha=0.2, label="Odchylenie standardowe"
    )

    # Rysowanie najlepszego i najgorszego fitness
    plt.plot(generations, max_fitness, label="Najlepszy fitness", color="green", linestyle="--")
    plt.plot(generations, min_fitness, label="Najgorszy fitness", color="red", linestyle="--")

    # Opis osi i tytuł
    plt.xlabel("Generacja")
    plt.ylabel("Fitness")
    plt.title("Ewolucja fitness w trakcie eksperymentu")
    plt.legend()
    plt.grid()

    # Zapis wykresu
    plt.savefig(os.path.join(output_path, "fitness_evolution.png"))
    plt.close()

In [16]:
def plot_histogram(data, column, output_path):
    """
    Generuje histogram dla wybranego parametru (np. trafionych celów lub czasu).
    """
    plt.figure(figsize=(8, 6))
    sns.histplot(data[column], kde=True, bins=20)
    plt.title(f"Histogram wartości: {column}")
    plt.xlabel(column)
    plt.ylabel("Liczba rozwiązań")
    plt.grid()

    # Zapis wykresu
    plt.savefig(os.path.join(output_path, f"{column}_histogram.png"))
    plt.close()

In [22]:
def main():
    # Ścieżka do folderu z danymi
    experiment_folder = "./"
    output_path = "./"
    os.makedirs(output_path, exist_ok=True)

    # Analiza danych
    data = analyze_experiment(experiment_folder)

    # Generowanie wykresów
    plot_fitness_evolution(data, output_path)
    plot_histogram(data, "Collected Targets", output_path)
    plot_histogram(data, "Time", output_path)

    print(f"Wykresy zapisane w folderze: {output_path}")

In [24]:
main()

Wykresy zapisane w folderze: ./
